# Proyecto Herramientas II - Grupo Optimización

Primeramente se presentan las librerías que se utilizaron y que son necesarias para ejecutar el código. Se deja como comentarios la instalación de las mismas por si alguien ocupara instalar alguna.

In [175]:
# !pip install numpy
# !pip install pandas
# !pip install numexpr
# !pip install numba
# !pip install matplotlib
# !pip install seaborn
# !pip install joblib
# !pip install IPython

In [2]:
# Se cargan las librerías más comúnes que se van a usar
import numpy as np
import pandas as pd
import timeit # La librería timeit no requiere ser instalada

Se usó la librería timeit para determinar el tiempo de ejecución de los ejercicios.

Referencias:

https://docs.python.org/es/3/library/timeit.html

https://www.geeksforgeeks.org/timeit-python-examples/

## Ejercicio 1: Operaciones matriciales

In [3]:
# Se crea la matriz con la que se va a trabajar
matriz = np.random.rand(10_000, 10_000)

In [6]:
# Ahora se realizará el ejercicio usando la librería NumExpr
import numexpr as ne

# Se crea un string con el código que se quiere ejecutar
codigo = """
resultado = ne.evaluate('(matriz ** 100) * 10 + 5')
"""

# Se ejecuta el código 10 veces usando timeit y se almacenan los tiempos en una lista
tiempo_total = timeit.timeit(stmt = codigo, number = 10, globals = globals())

# Se calcula el tiempo promedio
tiempo_promedio = tiempo_total / 10

tiempo_promedio

2.3868186399999103

In [5]:
# Seguidamente se hace lo mismo usando la librería Numba
import numba as nb

# Se crea una función que realice la operación que queramos, agregándole decoradores para que numba compile esta función de manera eficiente 
@nb.njit(parallel = True)
def operacion(matriz):
    
    matriz_resultado = np.empty_like(matriz)
    
    for i in nb.prange(matriz.shape[0]):
        
        for j in range(matriz.shape[1]):
            
            matriz_resultado[i, j] = (matriz[i, j] ** 100) * 10 + 5
            
    return matriz_resultado

codigo_ej_1 = """
resultado = operacion(matriz)
"""

tiempo_total_ej_1 = timeit.timeit(stmt = codigo_ej_1, number = 10, globals = globals())

tiempo_promedio_ej_1 = tiempo_total_ej_1 / 10

tiempo_promedio_ej_1

0.7331014900002628

In [8]:
# Por último, se crea un dataframe con el tiempo promedio (en segundos) obtenido por cada uno de los integrantes usando cada librería

data_ej_1 = {

    'Integrante': ['Eyeri', 'Alejandro'],

    'numexpr': [0.581, 2.387],

    'numba': [0.218, 0.733]
    
}

tiempos_matrices_ej_1 = pd.DataFrame(data_ej_1)

tiempos_matrices_ej_1

,Integrante,numexpr,numba
0,Eyeri,0.581,0.218
1,Alejandro,2.387,0.733


## Ejercicio 2: Limpieza de bases de datos

In [18]:
import pandas as pd
import numpy as np
import timeit

def limpiar_datos():
    muertes_cr = pd.read_excel('data/muertes_en_costa_rica_2014_2021.xlsx',
                               engine='openpyxl')
    
    muertes_cr = muertes_cr.drop(columns=['pc', 'causamuer', 'des_causa', 'instmurio', 'pcocu', 'nacmadre', 'pcregis', 'gruposcb'])
    
    # Se realiza la limpieza de la base de datos
    
    muertes_cr = muertes_cr[muertes_cr['edads'] >= 15]
    
    muertes_cr = muertes_cr[muertes_cr['anodef'] >= 2014]
    
    muertes_cr = muertes_cr[muertes_cr['anotrab'] >= 2014]
    
    muertes_cr = muertes_cr[muertes_cr['anodeclara'] >= 2014]
    
    muertes_cr['estcivil'] = muertes_cr['estcivil'].str.replace("Ã³", "o")
    
    muertes_cr['ocuparec'] = muertes_cr['ocuparec'].str.replace("Ã¡", "a")
    
    muertes_cr['ocuparec'] = muertes_cr['ocuparec'].str.replace("Ã©", "e")
    
    muertes_cr['ocuparec'] = muertes_cr['ocuparec'].str.replace("Ã", "i")
    
    muertes_cr['regsalud'] = muertes_cr['regsalud'].str.replace("Ã\xad", "i")
    
    muertes_cr['regsalud'] = muertes_cr['regsalud'].str.replace("Ã³", "o")
    
    muertes_cr['provincia'] = muertes_cr['provincia'].str.replace("Ã©", "e")
    
    muertes_cr['provincia'] = muertes_cr['provincia'].str.replace("Ã³", "o")
    
    muertes_cr['provocu'] = muertes_cr['provocu'].str.replace("Ã©", "e")
    
    muertes_cr['provocu'] = muertes_cr['provocu'].str.replace("Ã³", "o")
    
    muertes_cr['provregis'] = muertes_cr['provregis'].str.replace("Ã©", "e")
    
    muertes_cr['provregis'] = muertes_cr['provregis'].str.replace("Ã³", "o")
    
    muertes_cr['reginec'] = muertes_cr['reginec'].str.replace("Ã\xad", "i")
    
    muertes_cr['reginec'] = muertes_cr['reginec'].str.replace("Ã³", "o")
    
    muertes_cr['edadsrec'] = muertes_cr['edadsrec'].str.replace("100 y mÃ¡s", "100 - 121")
    
    muertes_cr['autopsia'] = muertes_cr['autopsia'].str.replace("Ã©", "e")
    
    muertes_cr['autopsia'] = muertes_cr['autopsia'].str.replace("Ã\xad", "i")
    
    muertes_cr['asistmed'] = muertes_cr['asistmed'].str.replace("Ã©", "e")
    
    muertes_cr['asistmed'] = muertes_cr['asistmed'].str.replace("Ã\xad", "i")
    
    muertes_cr['nacionalid'] = muertes_cr['nacionalid'].apply(lambda x: 'Extranjero' if x != 'Costa Rica' else x)
    
    otros = ['Ignorado', 'Union libre', 'Separado', 'Menor']
    
    muertes_cr['estcivil'] = muertes_cr['estcivil'].replace(otros, 'Otros')
    
    trabajadores_activos = ['Profesionales cienti\xadficos e intelectuales', 'Agricultores y trabajadores calificados agropecuarios, forestales y pesqueros',
                            'Ocupaciones elementales', 'Trabajadores de los servicios y vendedores de comercios y mercados',
                            'Operadores de instalaciones y maquinas y ensambladores', 'Tecnicos y profesionales de nivel medio',
                            'Oficiales, operarios y artesanos de artes mecanicas y de otros oficios', 'Personal de apoyo administrativo',
                            'Directores y gerentes']
    
    muertes_cr['ocuparec'] = muertes_cr['ocuparec'].replace(trabajadores_activos, 'Trabajadores activos')
    
    otros = ['Pensionado', 'Persona con discapacidad', 'Estudiante', 'Mal especificadas', 'Privado de libertad']
    
    muertes_cr['ocuparec'] = muertes_cr['ocuparec'].replace(otros, 'Otros')
    
    rangos_etarios = ["15 - 19", "20 - 24", "25 - 29", "30 - 34", "35 - 39", "40 - 44", "45 - 49", 
                      "50 - 54", "55 - 59", "60 - 64", "65 - 69", "70 - 74", "75 - 79", "80 - 84", 
                      "85 - 89", "90 - 94", "95 - 99", "100 - 121"]
    
    muertes_cr['edadsrec'] = pd.Categorical(muertes_cr['edadsrec'], categories = rangos_etarios, ordered = True)
    
    muertes_cr.reset_index(drop = True, inplace = True)
    
    return muertes_cr
    

tiempo_total_ej_2 = timeit.timeit(stmt=limpiar_datos, number=10)

tiempo_promedio_ej_2 = tiempo_total_ej_2 / 10

tiempo_promedio_ej_2


109.16192563000004

In [19]:
data_ej_2 = {

    'Integrante': ['Alejandro', ],

    'tiempo_promedio': [109.162, ]
    
}

tiempos_graficos_ej_2 = pd.DataFrame(data_ej_2)
tiempos_graficos_ej_2

,Integrante,tiempo_promedio
0,Alejandro,109.162


## Ejercicio 3: Gráficos en paralelo 

Primeramente se usa joblib para paralelizar la generación de los histogramas y de los gráficos de barras, por medio de los componentes Parallel y delayed

Referencias:

https://joblib.readthedocs.io/en/latest/parallel.html

https://joblib.readthedocs.io/en/latest/generated/joblib.Parallel.html

También se hace uso de la función clear_output del módulo IPython, con el fin de limpiar la salida de la celda y no mostrar todos los gráficos
generados cada vez que se ejecuta la función de generar gráficos

Referencias:

https://stackoverflow.com/questions/24816237/ipython-notebook-clear-cell-output-in-code

https://notebook.community/CestDiego/emacs-ipython-notebook/tests/notebook/nbformat4/Animations%20Using%20clear_output

In [43]:
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed
from IPython.display import clear_output

# Se define una función que realice los histogramas para las variables numéricas
def histograma(df, columna):
    
    plt.figure()
    
    sns.histplot(df[columna], bins = 'auto', color = 'blue', edgecolor = 'black')
    
    plt.xlabel('Valor')
    
    plt.ylabel('Frecuencia')
    
    return plt.gcf()

# Se define otra función que haga los gráficos de barras para las variables categóricas
def barras(df, columna):
    
    plt.figure()
    
    sns.countplot(x = columna, data = df, color = 'red')

    plt.xticks(rotation = 45)
    
    plt.xlabel('Categorías')
    
    plt.ylabel('Cantidad')
    
    return plt.gcf()

# Definimos una función que devuelve las columnas numéricas y categóricas en listas por aparte
def tipos_columnas(df):
    
    numericas = df.select_dtypes(include = ['number']).columns
        
    categoricas = df.select_dtypes(include = ['object', 'category']).columns

    return numericas, categoricas

# Obtenemos las variables numericas y categóricas
numericas, categoricas = tipos_columnas(muertes_cr)

# Definimos una función que genere los gráficos deseados
def generar_graficos(limpiar = True):

    # Generamos y guardamos los gráficos tanto de histogramas como de barras usando la paralelización de joblib
    histogramas = Parallel(n_jobs = -1)(
    
        delayed(histograma)(muertes_cr, col) for col in numericas
    
    )

    graficos_barras = Parallel(n_jobs = -1)(
    
        delayed(barras)(muertes_cr, col) for col in categoricas
    
    )

    # Mostramos los gráficos
    for grafico in histogramas + graficos_barras:

        plt.show()

    # Si el parámetro limpiar es True, no se mostrarán los gráficos
    if limpiar:

        clear_output()

# Así, se mide el tiempo que tarda la función al ejecutarla 10 veces
tiempo_ejecucion_ej_3 = timeit.timeit(generar_graficos, number = 10)

# Se obtiene el tiempo promedio de las 10 ejecuciones
tiempo_promedio_ej_3 = tiempo_ejecucion_ej_3 / 10
print(tiempo_promedio_ej_3)



In [44]:
# Por último, se crea un dataframe con el tiempo promedio que tardó la función por cada integrante
data_ej_3 = {

    'integrante': ['Eyeri', 'Alejandro'],

    'tiempo_promedio': [8.82, 9.71]
    
}

tiempos_graficos_ej_3 = pd.DataFrame(data_ej_3)
tiempos_graficos_ej_3

,integrante,tiempo_promedio
0,Eyeri,8.82
1,Alejandro,9.71


## Ejercicio 4: media movil 

In [25]:
import pandas as pd
import numpy as np
from numba import njit, prange

@njit(parallel=True)
def imputar_valores_nulos(data, banda):
    n = len(data)
    resultado = data.copy()
    
    for i in prange(n):
        if np.isnan(data[i]):
            principio = max(0, i - banda)
            final = min(n, i + banda + 1)
            suma, cuenta = 0.0, 0
            for j in range(principio, final):
                if not np.isnan(data[j]):
                    suma += data[j]
                    cuenta += 1
            if cuenta > 0:
                resultado[i] = suma / cuenta
    return resultado

def imputar_por_prom_movil(dataframe, columna, banda):

    data = dataframe[columna].values
    data_imputado = imputar_valores_nulos(data, banda)
    dataframe[columna] = data_imputado
    return dataframe


In [30]:
base_salarios = pd.read_excel('data/Base_salarios.xlsx',
                               engine='openpyxl')
codigo_ej_4 = """
resultado = imputar_por_prom_movil(base_salarios, 'Salario base', 4)
"""

tiempo_total_ej_4 = timeit.timeit(stmt = codigo_ej_4, number = 10, globals = globals())

tiempo_promedio_ej_4 = tiempo_total_ej_4 / 10

tiempo_promedio_ej_4


0.0003112099999270868

In [33]:
data_ej_4= {

'Integrante': ['Alejandro'],
'Tiempo promedio' : [0.0003, ]
}

tiempos_graficos_ej_4 = pd.DataFrame(data_ej_4)
tiempos_graficos_ej_4

,Integrante,Tiempo promedio
0,Alejandro,0.0003


## Ejercicio 5: eliminar columna por porcentaje de nulos 

In [9]:
import pandas as pd

def eliminar_columnas_por_nulos(df, porcentaje):
    # Calculamos el número máximo de valores nulos permitidos
    max_nulos = len(df) * porcentaje
    
    # Filtramos las columnas que tienen menos de max_nulos valores nulos
    df_filtrado = df.dropna(axis=1, thresh=len(df) - max_nulos)
    
    return df_filtrado

In [31]:

codigo_ej_5 = """
base_salarios_filtrada = eliminar_columnas_por_nulos(base_salarios, porcentaje=0.01)
"""

tiempo_total_ej_5 = timeit.timeit(stmt = codigo_ej_5, number = 10, globals = globals())

tiempo_promedio_ej_5 = tiempo_total_ej_5 / 10

tiempo_promedio_ej_5


0.007145939999827533

In [34]:
data_ej_5 = {

'Integrante': ['Alejandro'],
'Tiempo promedio' : [0.0071, ]
}

tiempos_graficos_ej_5 = pd.DataFrame(data_ej_5)
tiempos_graficos_ej_5

,Integrante,Tiempo promedio
0,Alejandro,0.0071


## Ejercicio 6: imputacion por agrupacion


In [39]:
# Función para imputar valores nulos por agrupación
def imputar_por_agrupacion():
    # Imputación utilizando pandas
    base_salarios['Salario base'] = base_salarios.groupby(['Género', 'Grado de estudio'])['Salario base'] \
                           .transform(lambda x: x.fillna(x.mean()))
    return base_salarios

In [41]:
# Así, se mide el tiempo que tarda la función al ejecutarla 10 veces
tiempo_ejecucion_ej_6 = timeit.timeit(imputar_por_agrupacion, number = 10)

# Se obtiene el tiempo promedio de las 10 ejecuciones
tiempo_promedio_ej_6 = tiempo_ejecucion_ej_6 / 10
print(tiempo_promedio_ej_6)

0.06680876999998872


In [42]:
data_ej_6 = {

'Integrante': ['Alejandro'],
'Tiempo promedio' : [0.0668, ]
}

tiempos_graficos_ej_6 = pd.DataFrame(data_ej_6)
tiempos_graficos_ej_6


,Integrante,Tiempo promedio
0,Alejandro,0.0668
